In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

In [8]:
%%sql
SELECT * 
FROM appearances
LIMIT 2

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
2 rows affected.


yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
1871,TRO,NA,abercda01,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1871,RC1,NA,addybo01,25,25,25,25,0,0,0,22,0,3,0,0,0,0,0,0,0


In [9]:
%%sql
SELECT * 
FROM batting
LIMIT 2

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
2 rows affected.


playerid,yearid,stint,teamid,lgid,g,ab,r,h,h2b,h3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
abercda01,1871,1,TRO,NA,1,4,0,0,0,0,0,0,0,0,0,0,None,None,None,None,0
addybo01,1871,1,RC1,NA,25,118,30,32,6,0,0,13,8,1,4,0,None,None,None,None,0


In [10]:
%%sql
SELECT * 
FROM collegeplaying
LIMIT 2

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
2 rows affected.


playerid,schoolid,yearid
aardsda01,pennst,2001
aardsda01,rice,2002


In [11]:
%%sql
SELECT *
FROM schools
LIMIT 2

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
2 rows affected.


schoolid,schoolname,schoolcity,schoolstate,schoolnick
abilchrist,Abilene Christian University,Abilene,TX,USA
adelphi,Adelphi University,Garden City,NY,USA


## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

**The first numbers you see after cost= are known as the “startup cost”. This is an estimate of how long it will take to fetch the first row. As such, the startup cost of an operation includes the cost of its children.

In [12]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [13]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [14]:
%%sql
SELECT namefirst, namelast, halloffame.playerid, yearid
FROM halloffame
INNER JOIN people ON people.playerid = halloffame.playerid
LIMIT 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


namefirst,namelast,playerid,yearid
Hank,Aaron,aaronha01,1982
Jim,Abbott,abbotji01,2005
Babe,Adams,adamsba01,1937
Babe,Adams,adamsba01,1938
Babe,Adams,adamsba01,1939
Babe,Adams,adamsba01,1942
Babe,Adams,adamsba01,1945
Babe,Adams,adamsba01,1946
Babe,Adams,adamsba01,1947
Babe,Adams,adamsba01,1948


In [15]:
%%sql result_1a <<
SELECT namefirst, namelast, halloffame.playerid, yearid
FROM halloffame
JOIN people ON people.playerid = halloffame.playerid AND inducted = 'Y'

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [16]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [17]:
pd.read_csv('results/result_1a.csv').shape

(323, 4)

In [18]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [19]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [20]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=10.865..11.628 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.030..0.655 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=10.604..10.606 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.010..5.172 rows=19370 loops=1)
 Planning Time: 1.137 ms
 Execution Time: 11.797 ms
(10 rows)



In [21]:
inducted_hof_query_str = "SELECT namefirst, namelast, halloffame.playerid, yearid FROM halloffame JOIN people ON people.playerid = halloffame.playerid AND inducted = 'Y'"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=10.461..11.187 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (people.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.029..0.594 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=10.215..10.216 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.010..5.069 rows=19370 loops=1)
 Planning Time: 1.063 ms
 Execution Time: 11.312 ms
(10 rows)



In [23]:
provided_query_cost = 959.06
provided_query_timing = 11.797
your_query_cost = 959.06
your_query_timing = 11.312

In [24]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [25]:
q1b_part2 = ['A', 'F']

In [26]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

idea: successfully inducted into the hall of fame and played in college at a school located in california

In [27]:
%%sql 
SELECT namefirst, namelast, c.playerid, c.schoolid, hof.yearid
FROM collegeplaying AS c, (SELECT namefirst, namelast, p.playerid, yearid
    FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
    WHERE p.playerid = hof.playerid) AS hof, (SELECT * FROM schools WHERE schoolstate = 'CA') AS sch
                                             
WHERE c.playerid = hof.playerid AND sch.schoolid = c.schoolid
ORDER BY hof.yearid, c.playerid
LIMIT 2;


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
2 rows affected.


namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971


In [28]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS 
SELECT namefirst, namelast, c.playerid, c.schoolid, hof.yearid
FROM collegeplaying AS c, (SELECT namefirst, namelast, p.playerid, yearid
    FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
    WHERE p.playerid = hof.playerid) AS hof, (SELECT * FROM schools WHERE schoolstate = 'CA') AS sch
                         
WHERE c.playerid = hof.playerid AND sch.schoolid = c.schoolid
ORDER BY hof.yearid, c.playerid;

SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [29]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [30]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [31]:
%%sql with_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca as hof
GROUP BY schoolid, yearid
ORDER BY yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [32]:
%%sql without_view <<
SELECT s.schoolid, h.yearid, COUNT(*)
FROM collegeplaying c, halloffame h, schools s
WHERE s.schoolstate = 'CA' AND h.inducted = 'Y' AND c.schoolid = s.schoolid AND c.playerid = h.playerid
GROUP BY s.schoolid, h.yearid
ORDER BY yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [33]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [34]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [35]:
with_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca as hof GROUP BY schoolid, yearid ORDER BY yearid;"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=520.03..520.69 rows=33 width=20) (actual time=10.237..10.255 rows=13 loops=1)
   Group Key: hof.yearid, hof.schoolid
   ->  Sort  (cost=520.03..520.11 rows=33 width=12) (actual time=10.229..10.234 rows=23 loops=1)
         Sort Key: hof.yearid, hof.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on hof  (cost=518.79..519.20 rows=33 width=12) (actual time=10.158..10.169 rows=23 loops=1)
               ->  Sort  (cost=518.79..518.87 rows=33 width=257) (actual time=10.156..10.162 rows=23 loops=1)
                     Sort Key: halloffame.yearid, c.playerid
                     Sort Method: quicksort  Memory

In [37]:
without_view_query_str = "SELECT s.schoolid, h.yearid, COUNT(*) FROM collegeplaying c, halloffame h, schools s WHERE s.schoolstate = 'CA' AND h.inducted = 'Y' AND c.schoolid = s.schoolid AND c.playerid = h.playerid GROUP BY s.schoolid, h.yearid ORDER BY yearid;"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                    QUERY PLAN                                                                     
---------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=488.95..490.87 rows=96 width=20) (actual time=6.213..6.226 rows=13 loops=1)
   Group Key: h.yearid, s.schoolid
   ->  Sort  (cost=488.95..489.19 rows=96 width=12) (actual time=6.204..6.209 rows=23 loops=1)
         Sort Key: h.yearid, s.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Hash Join  (cost=386.42..485.79 rows=96 width=12) (actual time=5.731..6.160 rows=23 loops=1)
               Hash Cond: ((h.playerid)::text = (c.playerid)::text)
               ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=323 width=13) (actual time=0.028..0.667 rows=323 loops=1)
                     Filter: ((inducted)::text = 'Y'::text)
                     Rows Rem

In [39]:
with_view_cost = 520.69
with_view_timing = 10.496
without_view_cost = 490.87
without_view_timing = 6.364

In [40]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [41]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT namefirst, namelast, c.playerid, c.schoolid, hof.yearid
FROM collegeplaying AS c, (SELECT namefirst, namelast, p.playerid, yearid
    FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
    WHERE p.playerid = hof.playerid) AS hof, (SELECT * FROM schools WHERE schoolstate = 'CA') AS sch
                         
WHERE c.playerid = hof.playerid AND sch.schoolid = c.schoolid
ORDER BY hof.yearid, c.playerid;

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [42]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [43]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [44]:
%%sql with_materialized_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca_mat as hof
GROUP BY schoolid, yearid
ORDER BY yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [45]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [46]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [47]:
with_materialized_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca_mat as hof GROUP BY schoolid, yearid ORDER BY yearid;"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                            QUERY PLAN                                                            
----------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.073..0.075 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.022..0.026 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat hof  (cost=0.00..12.30 rows=230 width=52) (actual time=0.004..0.006 rows=23 loops=1)
 Planning Time: 0.322 ms
 Execution Time: 0.180 ms
(9 rows)



In [48]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.210

In [49]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [50]:
q1e_part4 = ['D', 'J']

In [51]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

Views are different from Materialized Views in that they are calculated on demand, making their use in subsequent queries longer if not the same, they do not reduce execution time or cost because the views are calculated on demand every time the view is called. However, materialized views will reduce the cost and execution of a query because materialized views are only calculated once and are pretty much pre computed when calling these views in subsequent queries which reduces the cost and execution of these. However, since Materialized Views actually stores the data, these take more time to create compared to typical views.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [52]:
query_view_str = "SELECT * FROM inducted_hof_ca"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                    QUERY PLAN                                                                     
---------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=528.21..528.30 rows=33 width=33) (actual time=6.253..6.258 rows=23 loops=1)
   Sort Key: halloffame.yearid, c.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=386.71..527.38 rows=33 width=33) (actual time=5.556..6.197 rows=23 loops=1)
         Join Filter: ((halloffame.playerid)::text = (p.playerid)::text)
         ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=5.500..5.918 rows=23 loops=1)
               Hash Cond: ((halloffame.playerid)::text = (c.playerid)::text)
               ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.033..0.749 rows=323 loops=1)
                     Filter: ((inducted)::text = '

In [54]:
query_view_cost = 528.30
query_view_timing = 6.432

In [55]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [56]:
%%sql result_2b <<
SELECT * 
FROM inducted_hof_ca
WHERE yearid > 2010

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [57]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010;"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=209.85..209.85 rows=2 width=33) (actual time=0.927..0.929 rows=8 loops=1)
   Sort Key: halloffame.yearid, c.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..209.84 rows=2 width=33) (actual time=0.572..0.888 rows=8 loops=1)
         Join Filter: ((halloffame.playerid)::text = (p.playerid)::text)
         ->  Nested Loop  (cost=0.56..207.24 rows=6 width=30) (actual time=0.540..0.807 rows=8 loops=1)
               ->  Nested Loop  (cost=0.29..190.63 rows=55 width=30) (actual time=0.526..0.708 rows=27 loops=1)
                     ->  Seq Scan on halloffame  (cost=0.00..106.86 rows=21 width=13) (actual time=0.481..0.521

In [59]:
query_view_with_filter_cost = 209.85
query_view_with_filter_timing = 1.066

In [60]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [61]:
q2c = ['A', 'E']

In [62]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Filtering data means that there is less data for the system to go through. Since there was only 8 rows that fit the criteria of > 2010, there was fewer rows to be dealt with when calling the view since the filter got applied when the view was created on demand because of predicate pushdown. Since filtering reduced the data so much, we also didn't need to do a hash join and nested loop join was both faster and less costly. 

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [63]:
%%sql q3a_join <<
SELECT * 
FROM people p
JOIN collegeplaying c ON c.playerid = p.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [64]:
q3a_query_str = "SELECT * FROM people p JOIN collegeplaying c ON c.playerid = p.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=8.496..18.430 rows=17350 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.021..1.373 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=8.270..8.272 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.016..1.687 rows=19370 loops=1)
 Planning Time: 1.081 ms
 Execution Time: 19.189 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [65]:
q3a = ["C"]

In [66]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [67]:
%%sql q3b_join <<
SELECT * 
FROM people p
JOIN collegeplaying c ON c.playerid = p.playerid
ORDER BY c.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [68]:
q3b_query_str = "SELECT * FROM people p JOIN collegeplaying c ON c.playerid = p.playerid ORDER BY c.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                         QUERY PLAN                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..1910.20 rows=17350 width=167) (actual time=0.040..17.553 rows=17350 loops=1)
   Merge Cond: ((p.playerid)::text = (c.playerid)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..1024.36 rows=19370 width=146) (actual time=0.018..4.091 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying c  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.017..2.242 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 1.022 ms
 Execution Time: 18.265 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [69]:
q3b = ["B"]

In [70]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [71]:
%%sql q3c_join <<
SELECT * 
FROM people p
JOIN people p2 ON p2.playerid != p.playerid
LIMIT 1000;

Inspect the query plan for the query you wrote above.

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


In [72]:
q3c_query_str = "SELECT * FROM people p JOIN people p2 ON p2.playerid != p.playerid LIMIT 1000;"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.052..1.039 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.051..0.967 rows=1000 loops=1)
         Join Filter: ((p2.playerid)::text <> (p.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.019..0.019 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.026..0.478 rows=1001 loops=1)
               ->  Seq Scan on people p2  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.001..0.090 rows=1001 loops=1)
 Planning Time: 1.012 ms
 Execution Time: 1.228 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [73]:
q3c = ['A']

In [74]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

In 3a, when performing an inner join, the query optimizer chose hash join because the joined tables are not sorted (making merge sort not an option) and given that there are a lot of rows to go through, the nested loop would be too CPU intensive.

In 3b, when performing an inner join and then sorting, the query optimer used merge join because merge join returns data in sorted order which has been specified by the order by statement, but also requires all input to be sorted by the join columns. We have done this on each table by sorting on playerid. In this case merge sort would reduce the time it would take to join using another method and then sort through each row by playerid because it would only have to go through each table once.

In 3d when performing a cross join to get every single combination, the optimizer chose a nested loop join since we start with one value of our outer table and iterate over every value of our inner table comparing values each time. In nested loop join, we compare every row in the first table with every row in the second table which is necessary for every single possible combinations of two players as we are essentially doing a cross product. We are also limiting the rows to 1000 which reduces the strain that this nested loop join would have on CPU making it a good option for this problem.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [75]:
%%sql result_4a <<
SELECT app.playerid, SUM(salary)
FROM (SELECT * FROM appearances WHERE g_batting = 10) AS app
NATURAL JOIN salaries
GROUP BY playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [76]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [77]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [98]:
result_4a_query_str = "SELECT app.playerid, SUM(salary) FROM (SELECT * FROM appearances WHERE g_batting = 10) AS app NATURAL JOIN salaries GROUP BY playerid;"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3635.14..3635.16 rows=1 width=17) (actual time=20.262..20.314 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=3635.14..3635.15 rows=1 width=17) (actual time=20.257..20.264 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2898.36..3635.13 rows=1 width=17) (actual time=12.631..20.116 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salar

In [100]:
result_4a_cost = 3635.16
result_4a_timing = 20.405

In [101]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [102]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [103]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2350.86..2350.88 rows=1 width=17) (actual time=10.360..10.409 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=2350.86..2350.87 rows=1 width=17) (actual time=10.354..10.362 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1614.08..2350.85 rows=1 width=17) (actual time=3.016..10.213 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salari

In [105]:
result_4b_cost = 2350.88
result_4b_timing = 10.502

In [106]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [107]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [108]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries (salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [109]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3635.14..3635.16 rows=1 width=17) (actual time=20.421..20.476 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=3635.14..3635.15 rows=1 width=17) (actual time=20.414..20.423 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2898.36..3635.13 rows=1 width=17) (actual time=12.062..20.264 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salar

In [112]:
result_4c_cost = 3635.16
result_4c_timing = 20.587

In [113]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [115]:
q4d_part1 = ['D', 'E']

In [116]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

Adding indexes helps the system look for data matching certain characteristics. Adding an index to the g_batting column decreased cost and time because we were looking for tuples that had g_batting = 10, so it sped that process up using an index scan method. Indexes can also speed up joins but in this case, since we were indexing on the salary column, there was no change to cost or execution because we were not joining on the salary column as it was exclusive to the salaries table the query plan has no use for that index.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [117]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [118]:
%%sql result_4ei <<
SELECT appearances.playerid, appearances.yearid, salary
FROM (SELECT * FROM appearances WHERE g_all=10 AND g_batting = 10) as appearances
NATURAL JOIN salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [119]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [120]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [121]:
result_4ei_query_str = "SELECT playerid, yearid, salary FROM (SELECT * FROM appearances WHERE g_all=10 AND g_batting = 10) as appearances NATURAL JOIN salaries;"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3279.47 rows=1 width=21) (actual time=8.521..15.514 rows=120 loops=1)
   ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=18 width=20) (actual time=0.017..12.316 rows=1289 loops=1)
         Filter: ((g_all = 10) AND (g_batting = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries  (cost=0.29..8.09 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((yearid = appearances.yearid) AND ((teamid)::text = (appearances.teamid)::text) AND ((lgid)::text = (appearances.lgid)::text) AND ((pla

In [123]:
result_4ei_cost = 3279.47
result_4ei_timing = 15.579

In [124]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [125]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [126]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.02..1737.38 rows=1 width=21) (actual time=4.169..7.393 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=17.73..1591.75 rows=18 width=20) (actual time=0.397..3.297 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..17.73 rows=1258 width=0) (actual time=0.235..0.236 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries  (cost=0.

In [128]:
result_4eii_with_index_cost = 1793.38
result_4eii_with_index_timing = 7.489

In [129]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [130]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [131]:
%%sql result_4eiii <<
SELECT appearances.playerid, appearances.yearid, salary
FROM (SELECT * FROM appearances WHERE g_all=10 OR g_batting = 10) as appearances
NATURAL JOIN salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [132]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [133]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [134]:
result_4eiii_query_str = "SELECT appearances.playerid, appearances.yearid, salary FROM (SELECT * FROM appearances WHERE g_all=10 OR g_batting = 10) as appearances NATURAL JOIN salaries;"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3188.86..3925.63 rows=1 width=21) (actual time=13.128..20.212 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.014..1.676 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2751 width=20) (actual time=12.920..12.921 rows=

In [135]:
result_4eiii_cost = 3933.31
result_4eiii_timing = 22.344

In [136]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [137]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [138]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3188.86..3925.63 rows=1 width=21) (actual time=14.831..22.246 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.014..1.778 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2751 width=20) (actual time=14.629..14.630 rows=

In [139]:
result_4eiv_with_index_cost = 3933.31
result_4eiv_with_index_timing = 22.626 

In [140]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [141]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [142]:
# %%sql
# DROP INDEX g_batting_g_all_idx

In [143]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances (g_batting, g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [149]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2889.93..3626.70 rows=1 width=21) (actual time=3.942..11.155 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.014..1.728 rows=26428 loops=1)
   ->  Hash  (cost=2834.91..2834.91 rows=2751 width=20) (actual time=3.769..3.771 rows=165

In [145]:
result_4ev_multiple_col_index_cost = 3606.73
result_4ev_multiple_col_index_timing = 12.763

In [146]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [147]:
q4e_part6 = ['C', 'I']

In [148]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

Using indexes with an AND predicate like we did 4eii reduced both cost and time because in an AND predicate we can essentially only search through one column and remove tuples that don't hold (because by AND they shouldn't be returned), which reduces the amount of filtering the system has to do. Then the filtering on g_batting is also faster due to the Bitmap Heap Scan searching which limits the which is faster than sequential search over all the tuples. 

Using indexes on one column used in OR statements did not reduce time or cost because even though we can find tuples where g_batting = 10 using the index that we created for that specific column, we would still need to go through all the tuples again for the second column using sequential search for each column because even if g_batting was false, the g_all could be true. So then using index methods here would not be beneficial since we can't actually use any of the filtered true rows and the query plan just sticks to sequential search. 

Using indexes on both columns reduced both the query time and cost when used in OR statements because, despite this being an OR predicate, having two indexes means that we can look at only the indexes . We still have to search every single tuple but if we have a multicolumn index we can still narrow down the search range that corressponds to only those two indexes rather than searching through all the data pages. The query plan shows that there is a reduction in cost and execution time, but it is not as significant as using an AND predicate.

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [ ]:
%%sql result_5a_min << 
SELECT MIN(salary)
FROM salaries

In [ ]:
result_5a_min_query_str = "SELECT MIN(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

In [ ]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 4.568

In [ ]:
grader.check("q5ai")

In [ ]:
%%sql result_5a_avg <<
SELECT AVG(salary)
FROM salaries

In [ ]:
result_5a_avg_query_str = "SELECT AVG(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

In [ ]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 5.102

In [ ]:
grader.check("q5aii")

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [ ]:
%%sql 
CREATE INDEX salary_idx
ON salaries (salary);

In [ ]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

In [ ]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing = 0.136

In [ ]:
grader.check("q5bi")

In [ ]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

In [ ]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 8.508

In [ ]:
grader.check("q5bii")

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [ ]:
%%sql 
DROP INDEX salary_idx

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [ ]:
q5c = ['D']

In [ ]:
grader.check("q5c")

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

Looking at the query plans, we can see that using AVG or COUNT doesn't improve performance, and even with indexing these use the same query plan. However, looking at the MIN (or MAX) aggregation, this reduces both cost and execution. Count and Average are slower aggregation functions because they both must use a sequential scan as they must read every single tuple before their output is ready. This means that indexing COUNT or AVG columns does not improve any performance because we would still need to go through every tuple which defeats the purpose of having an index, and therefore we would still use the sequential scan. 

On the other hand, MIN or MAX are good functions to use with an indexed column because these queries can make use of index specific scans as they are only looking for single values. In these cases using the index would allow SQL to simply navigate to the first OR last leaf block in the index structure.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [ ]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500

In [ ]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [ ]:
grader.check("q6ai")

In [ ]:
result_6a_query_str = "SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [ ]:
result_6a_cost = 2878.05
result_6a_timing = 20.290

In [ ]:
grader.check("q6aii")

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [ ]:
%%sql 
CLUSTER batting USING batting_pkey

In [ ]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [ ]:
result_6b_cost = 2878.05
result_6b_timing = 15.412

In [ ]:
grader.check("q6b")

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [ ]:
# %%sql
# DROP INDEX ab_idx

In [ ]:
%%sql 
CREATE INDEX ab_idx
ON batting(ab);

In [ ]:
%%sql
CLUSTER batting USING ab_idx;

In [ ]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [ ]:
result_6c_cost = 1785.39
result_6c_timing = 2.306

In [ ]:
grader.check("q6c")

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [ ]:
q6d = ['A', 'D']

In [ ]:
grader.check("q6d")

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

Clustering based on the batting_pkey didn't have much of an effect on the execution time or cost of the query because since it the batting_pkey is a primary key, and therefore all unique, there was really not any added benefit (or detriment) to doing this. The number of clusters was equivalent to the number of batting_pkey's, additionally, the where clause is not using the primary key and therefore adding that specific clustering did not change anything about the query plan.

Clustering and indexing on ab from the batting table makes it such that instead of having to go through all 10k rows of the batting table, we only have to go through each value that ab can be (which is probably much smaller than 0-10k). Then we can use the index structure to optimize the where statement for ab > 500, which decreased both cost and time.

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [ ]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [ ]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [ ]:
result_7a_timing = 4

In [ ]:
grader.check("q7a")

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [ ]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [ ]:
%%sql 
CREATE INDEX salary_idx
ON salaries (salary);

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [ ]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [ ]:
result_7b_timing = 5.34

In [ ]:
grader.check("q7b")

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

After adding an index into the salaries table, the total time actually increased. It likely happened because updating the table by inserting many rows means that we needed to update the indexes as well. The database has to add each new entry to each index. So because 7b had an index on salary, this query took a little longer to run. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

1. Method 1: Adding an index to typically non mutable where claused attributes. This can optimize query performance because indexing makes attributes faster to be filtered and queried by creating pointers that point to where the data is stored. Indexing also allows for efficient search methods such as B+ trees which are usually more cost effective and much faster than the base sequential search, especially for large databases. However, indexing should probably be done on attributes that are not usually updated because of the time it takes to update the indexes.

2. Method 2: Pre filtering data can help significantly optimize query performance. This comes from the general idea that the less data the query has to work with, the less time and cost it will take to run. Pre filtering the data can lower the amount of data the system needs to run in a more intensive clause such as a join or group by.

3. Method 3: Merge join method (performed by the database system) optimizes query performance by first sorting the join attributes, the creating sorted runs of blocks, and then running a single pass through both sorted runs and merges. Additionally, merge join merges in sorted order which can be helpful. Since merge join only has to do a single pass through each table it is generally cheaper and more time efficient to do than nested loop or hash joins unless sorting hasn't already happened and the sorting would be expensive.

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [ ]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()